In [1]:
dbutils.library.installPyPI('snowflake-connector-python', version = "2.0.2")
dbutils.library.installPyPI("azure-storage-blob", version = "2.1.0")
dbutils.library.installPyPI("lightgbm")

dbutils.library.installPyPI("scikit-learn", version="0.18.2")

# dbutils.library.installPyPI('numpy', version="1.15.4")
dbutils.library.installPyPI('numpy', version="1.18.1")

dbutils.library.install("dbfs:/FileStore/jars/dash_ml-1.0.15-py3.6.egg")
# dbutils.library.installPyPI("dill", version="0.2.7.1")
dbutils.library.installPyPI("dill", version="0.2.9")

# dbutils.library.installPyPI("pathos")
dbutils.library.installPyPI("mlflow")
# dbutils.library.installPyPI("matplotlib", version = "1.5.3")
# dbutils.library.installPyPI("seaborn")
# dbutils.library.installPyPI("pdpbox")
# dbutils.library.installPyPI("causalml")
# dbutils.library.installPyPI("econml")
dbutils.library.installPyPI("category_encoders", version="2.2.2")
dbutils.library.installPyPI("pandas", version="0.21.0")


dbutils.library.restartPython()

PyPI package snowflake-connector-python has been installed already. The previous installed package is PyPI:(snowflake-connector-python)-(2.0.2)-(empty)-(empty). To resolve this issue detach and retach the notebook to create a new environment or rename the package.
PyPI package azure-storage-blob has been installed already. The previous installed package is PyPI:(azure-storage-blob)-(2.1.0)-(empty)-(empty). To resolve this issue detach and retach the notebook to create a new environment or rename the package.
PyPI package lightgbm has been installed already. The previous installed package is PyPI:(lightgbm)-(empty)-(empty)-(empty). To resolve this issue detach and retach the notebook to create a new environment or rename the package.
PyPI package scikit-learn has been installed already. The previous installed package is PyPI:(scikit-learn)-(0.18.2)-(empty)-(empty). To resolve this issue detach and retach the notebook to create a new environment or rename the package.
PyPI package numpy has been installed already. The previous installed package is PyPI:(numpy)-(1.18.1)-(empty)-(empty). To resolve this issue detach and retach the notebook to create a new environment or rename the package.
Library with this file name dbfs:/FileStore/jars/dash_ml-1.0.15-py3.6.egg already installed. If you would like to reinstall this please detach and reattach to recreate your environment. Conflict previous lib file: dbfs:/FileStore/jars/dash_ml-1.0.15-py3.6.egg
PyPI package dill has been installed already. The previous installed package is PyPI:(dill)-(0.2.9)-(empty)-(empty). To resolve this issue detach and retach the notebook to create a new environment or rename the package.
PyPI package mlflow has been installed already. The previous installed package is PyPI:(mlflow)-(empty)-(empty)-(empty). To resolve this issue detach and retach the notebook to create a new environment or rename the package.
PyPI package category_encoders has been installed already. The previous installed package is PyPI:(category_encoders)-(2.2.2)-(empty)-(empty). To resolve this issue detach and retach the notebook to create a new environment or rename the package.
PyPI package pandas has been installed already. The previous installed package is PyPI:(pandas)-(0.21.0)-(empty)-(empty). To resolve this issue detach and retach the notebook to create a new environment or rename the package.

In [2]:
import pandas as pd
import os
from os import path
from scipy.stats import pearsonr, spearmanr
from scipy import stats
import numpy as np
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.metrics import log_loss, accuracy_score, average_precision_score, confusion_matrix, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import snowflake.connector
import dill
from sklearn.pipeline import Pipeline
from category_encoders import TargetEncoder

In [3]:
pd.__version__
import sklearn
sklearn.__version__

Out[ 2 ]: '0.22.2.post1'

In [4]:
# class TargetEncoder:
#     def __init__(self, cols=None, n_folds=None):
#         # default to encoding all categorical columns
#         if isinstance(cols, str):
#             self.cols = [cols]
#         else:
#             self.cols = cols
#         self.n_folds = n_folds
#         self.kfold_means = dict()
#         self.target_means = dict()
#         self.fallback = None
#     def _reg_fit(self, X, y):
#         # Encode all categorical columns if not specified
#         if self.cols is None:
#             cols = []
#             for i, col in enumerate(X.dtypes):
#                 if col == 'object':
#                     cols.append(X.columns[i])
#             self.cols = cols
#         if self.n_folds is not None:
#             self.target_means = dict()
#         df = X.copy()
#         df['target'] = y.astype('float')
#         self.fallback = df['target'].mean()
#         for cat_col in self.cols:
#             target = df[[cat_col, 'target']].groupby(cat_col).mean().to_dict()['target']
#             self.target_means[cat_col] = target
#         return self
#     def _reg_transform(self, X, y):
#         X_transform = X.copy()
#         cols = []
#         ls_target_encodings = []
#         # Likely a way we can impute all the eligible columns without looping
#         # through each one
#         for col, val in self.target_means.items():
#             cols.append(col)
#             ls_target_encodings.append(
#                 X_transform[col].map(val).fillna(
#                     self.fallback))
#         X_transform[cols] = pd.concat(
#             ls_target_encodings,
#             axis='columns',
#             ignore_index=True)
#         return X_transform
#     def _kfold_numbering(self, X, y, n_folds):
#         # Is there a way I can vectorize this for loop?
#         df = X.copy()
#         df['target'] = y.astype('float')
#         parts = int(len(df) / n_folds)
#         kfold_df = []
#         for i in range(n_folds):
#             if i == n_folds:
#                 break
#             else:
#                 sliced_df = df[i * parts:(i + 1) * parts]
#                 sliced_df['fold'] = i
#                 kfold_df.append(sliced_df)
#         kfold_df = pd.concat(kfold_df)
#         return kfold_df
#     def _kfold_fit(self, X, y):
#         kfold_df = self._kfold_numbering(X, y, self.n_folds)
#         for i in range(self.n_folds):
#             # To Do: Figure out how to avoid re-computing these values in a few
#             # lines of code.
#             data_for_mean = kfold_df[kfold_df['fold'] != i]
#             data_for_mean_X = data_for_mean.drop(['target'], axis=1)
#             data_for_mean_y = data_for_mean['target']
#             self._reg_fit(data_for_mean_X, data_for_mean_y)
#             self.kfold_means[i] = self.target_means
#         return self
#     def _kfold_transform(self, X, y):
#         kfold_df = self._kfold_numbering(X, y, self.n_folds)
#         X_transform = []
#         for i in self.kfold_means.keys():
#             data_to_impute = kfold_df[kfold_df['fold'] == i]
#             data_to_impute_X = data_to_impute.drop(['target'], axis=1)
#             data_to_impute_y = data_to_impute['target']
#             # To Do: See if I can map to fold numbers to improve speed
#             self.target_means = self.kfold_means[i]
#             fold_transform_X = self._reg_transform(
#                 data_to_impute_X, data_to_impute_y)
#             X_transform.append(fold_transform_X)
#         return pd.concat(X_transform)
#     def fit(self, X, y):
#         if self.n_folds is not None:
#             assert self.n_folds < len(X)/self.n_folds, "Dataset not large enough to support specified number of folds"
#             assert self.n_folds < len(X)/self.n_folds, "Dataset not large enough to support specified number of folds"
#             self._kfold_fit(X, y)
#         self._reg_fit(X, y)
#     def transform(self, X, y):
#         if self.n_folds is not None:
#             return self._kfold_transform(X, y)
#         return self._reg_transform(X, y)
#     def fit_transform(self, X, y):
#         if self.n_folds is not None:
#             return self._kfold_fit(X, y, self.n_folds)._kfold_transform(X, y)
#         return self.fit(X, y).transform(X, y)

In [5]:
def eval_model(model, x_test, y_test):
  y_pred = model.predict(x_test)
  y_pred_proba = model.predict_proba(x_test)[:, 1]
  
  
  y_test_reshaped = np.reshape(y_test.astype('float').values, (len(y_test), ))
  
  print('in gt, pct of ones', y_test_reshaped.mean(), 'pct of zeros', 1-y_test_reshaped.mean())
  
  residual = y_pred_proba - y_test_reshaped
  print('residual std', residual.std(), 'y_test std', y_test_reshaped.std())
  
  logloss_test_pred = log_loss(y_test_reshaped, y_pred_proba)
#   logloss_baseline = log_loss(y_test_reshaped, np.ones(len(y_pred)))
  pred_random = np.random.randint(2, size=len(y_test_reshaped))
  logloss_baseline = log_loss(y_test_reshaped, pred_random)
  print('logloss_baseline', logloss_baseline)
  print('logloss_test_pred', logloss_test_pred)
  
  acc = accuracy_score(y_test_reshaped, y_pred)
  f1 = f1_score(y_test_reshaped, y_pred)  
  roc_auc = roc_auc_score(y_test_reshaped, y_pred_proba)
  print('acc', acc)
  print('average_precision_score predicted', average_precision_score(y_test_reshaped, y_pred_proba), 'average_precision_score all zeros', average_precision_score(y_test_reshaped, np.zeros(len(y_pred))))
  print('f1 score', f1)
  print('roc_auc_score', roc_auc)
  conf_max = confusion_matrix(y_test_reshaped, y_pred)/len(y_test_reshaped)
  print('conf mat', conf_max)
  
  return residual.std(), y_test_reshaped.std()

In [6]:
sql_query_train = """
with daypart_mapping as (
  SELECT 
    case when day_part = 'late_night' then 'latenight' 
      else DAY_PART
    END as daypart,
    MIN(LOCAL_HOUR) as min_hour,
    MAX(LOCAL_HOUR) as max_hour
  FROM PRODDB.STATIC.LOOKUP_DAY_PART_MAPPING 
  GROUP BY 1
),
flf_targets as (
SELECT 
  FLF.STARTING_POINT_ID, 
  FLF.STARTING_POINT_NAME, 
  FLF.TIME_OF_DAY as daypart, 
  dp.min_hour,
  dp.max_hour,
  FLF.TARGET_IDEAL_FLF, 
  FLF.MIN_TARGET_FLF_RANGE, 
  FLF.MAX_TARGET_FLF_RANGE, 
  FLF.TARGET_CREATED_AT, 
  LEAD(FLF.TARGET_CREATED_AT, 1) OVER (PARTITION BY FLF.STARTING_POINT_ID, FLF.TIME_OF_DAY ORDER BY FLF.TARGET_CREATED_AT) as next, 
  IFNULL(next, '2022-01-01') AS NEXT_TARGET_CREATED_DATE
FROM STATIC.LOOKUP_TARGET_FLF_BY_REGION flf
LEFT JOIN daypart_mapping dp 
  on flf.TIME_OF_DAY = dp.DAYPART
),
flf_raw as (
SELECT
  dd.created_at,
  dd.DELIVERY_ID,
  dd.active_date,
  dd.STORE_STARTING_POINT_ID,
  dd.SUBMARKET_ID,
  dd.flf,
  fces.num_delivered as num_delivered,
  fces.num_opened as num_opened,
  sm.LAUNCH_DATE as submarket_launch_date,
  convert_timezone('UTC', dd.TIMEZONE, dd.CREATED_AT) as created_at_local,
  TO_DATE(created_at_local) as created_at_local_date,
  hour(created_at_local) * 2 + floor(minute(created_at_local)/30.0) as window_id,
  datediff('second', dd.CREATED_AT, dd.ACTUAL_DELIVERY_TIME)/60.0 as asap,
  dd.DISTINCT_ACTIVE_DURATION/60.0 as dat,
  datediff('second', dd.DASHER_CONFIRMED_TIME, dd.DASHER_AT_STORE_TIME)/60.0 as d2r,
  case when datediff('second', dd.QUOTED_DELIVERY_TIME, dd.ACTUAL_DELIVERY_TIME)/60 > 20 then 1 else 0 END as lateness_20_min,
  flf.daypart,
  case when dd.flf - flf.MAX_TARGET_FLF_RANGE > 0 then 1 else 0 END as is_flf_above_max,
  case when dd.flf - flf.TARGET_IDEAL_FLF > 0 then 1 else 0 END as is_flf_above_ideal
FROM PRODDB.PUBLIC.DIMENSION_DELIVERIES dd 
LEFT JOIN flf_targets flf
  on dd.STORE_STARTING_POINT_ID = flf.STARTING_POINT_ID
  AND hour(convert_timezone('UTC', dd.TIMEZONE, dd.created_at)) between flf.min_hour and flf.max_hour
  AND convert_timezone('UTC', dd.TIMEZONE, dd.created_at) between flf.TARGET_CREATED_AT and flf.NEXT_TARGET_CREATED_DATE
LEFT JOIN PRODDB.PUBLIC.MAINDB_SUBMARKET sm 
  ON dd.SUBMARKET_ID = sm.ID
LEFT JOIN public.fact_cx_email_summary fces
  ON dd.SUBMARKET_ID = fces.SUBMARKET_ID
  AND convert_timezone('UTC',dd.timezone,
 dateadd('minute',cast(floor(date_part('minute',dd.created_at) / 30) * 30 as int), date_trunc('hour',dd.created_at))
 ) = fces.half_hour_local
WHERE dd.created_at between '2020-03-16' and '2020-04-27'
  AND dd.IS_FILTERED_CORE = true 
  AND dd.IS_ASAP = true 
  AND dd.IS_CONSUMER_PICKUP = false 
  AND fulfillment_type != 'merchant_fleet'
),

flf_raw_grouped as(
SELECT
    t.created_at_local_date,
    t.DAYPART,
    t.STORE_STARTING_POINT_ID,
    AVG(t.ASAP) as avg_asap,
    AVG(t.DAT) as avg_dat,
    AVG(t.D2R) as avg_d2r,
    AVG(t.IS_FLF_ABOVE_IDEAL) as avg_is_flf_above_ideal,
    AVG(t.flf) as avg_flf,
    AVG(t.num_opened) as avg_num_opened,
    AVG(t.num_delivered) as avg_num_delivered
FROM flf_raw t
GROUP BY t.created_at_local_date, t.DAYPART, t.STORE_STARTING_POINT_ID
),
flf_hist as(
SELECT 
    t1.created_at_local_date,
    dayofweek(t1.CREATED_AT_LOCAL) as DAY_OF_WEEK,
    hour(t1.CREATED_AT_LOCAL) as HOUR_OF_DAY,
    t1.STORE_STARTING_POINT_ID,
    t1.SUBMARKET_ID,  
    t1.DAYPART,
    t1.WINDOW_ID,
    t1.is_flf_above_ideal
FROM flf_raw t1
LEFT JOIN flf_raw_grouped t2
    ON t1.created_at_local_date = DATEADD(Day, -15, t2.created_at_local_date)
    AND t1.DAYPART = t2.DAYPART
    AND t1.STORE_STARTING_POINT_ID = t2.STORE_STARTING_POINT_ID
)

SELECT *
FROM flf_hist
SAMPLE(20)
"""

# sql_query_test_verify = """
# CREATE TEMP TABLE CHIZHANG.DAYPART_MAPPING AS (
#   SELECT 
#     CASE WHEN day_part = 'late_night' THEN 'latenight' 
#       ELSE DAY_PART
#     END AS daypart,
#     MIN(LOCAL_HOUR) AS min_hour,
#     MAX(LOCAL_HOUR) AS max_hour
#   FROM PRODDB.STATIC.LOOKUP_DAY_PART_MAPPING 
#   GROUP BY 1
# );

# CREATE temp TABLE CHIZHANG.FLF_TARGETS AS (
# SELECT 
#   FLF.STARTING_POINT_ID, 
#   FLF.STARTING_POINT_NAME, 
#   FLF.TIME_OF_DAY AS daypart, 
#   dp.min_hour,
#   dp.max_hour,
#   FLF.TARGET_IDEAL_FLF, 
#   FLF.MIN_TARGET_FLF_RANGE, 
#   FLF.MAX_TARGET_FLF_RANGE, 
#   FLF.TARGET_CREATED_AT, 
#   LEAD(FLF.TARGET_CREATED_AT, 1) OVER (PARTITION BY FLF.STARTING_POINT_ID, FLF.TIME_OF_DAY ORDER BY FLF.TARGET_CREATED_AT) AS NEXT, 
#   IFNULL(NEXT, '2022-01-01') AS NEXT_TARGET_CREATED_DATE
# FROM STATIC.LOOKUP_TARGET_FLF_BY_REGION flf
# LEFT JOIN CHIZHANG.DAYPART_MAPPING dp 
#   ON flf.TIME_OF_DAY = dp.DAYPART
# );
    
# CREATE temp TABLE CHIZHANG.FLF_RAW AS (
# SELECT
#   dd.created_at,
#   dd.DELIVERY_ID,
#   dd.active_date,
#   dd.STORE_STARTING_POINT_ID,
#   dd.SUBMARKET_ID,
#   dd.flf,
#   fces.num_delivered AS num_delivered,
#   fces.num_opened AS num_opened,
#   sm.LAUNCH_DATE AS submarket_launch_date,
#   convert_timezone('UTC', dd.TIMEZONE, dd.CREATED_AT) AS created_at_local,
#   TO_DATE(created_at_local) AS created_at_local_date,
#   hour(created_at_local) * 2 + floor(minute(created_at_local)/30.0) AS window_id,
#   datediff('second', dd.CREATED_AT, dd.ACTUAL_DELIVERY_TIME)/60.0 AS asap,
#   dd.DISTINCT_ACTIVE_DURATION/60.0 AS dat,
#   datediff('second', dd.DASHER_CONFIRMED_TIME, dd.DASHER_AT_STORE_TIME)/60.0 as d2r,
#   CASE WHEN datediff('second', dd.QUOTED_DELIVERY_TIME, dd.ACTUAL_DELIVERY_TIME)/60 > 20 THEN 1 ELSE 0 END AS lateness_20_min,
#   flf.daypart,
#   CASE WHEN dd.flf - flf.MAX_TARGET_FLF_RANGE > 0 THEN 1 ELSE 0 END AS is_flf_above_max,
#   CASE WHEN dd.flf - flf.TARGET_IDEAL_FLF > 0 THEN 1 ELSE 0 END AS is_flf_above_ideal
# FROM PRODDB.PUBLIC.DIMENSION_DELIVERIES dd 
# LEFT JOIN CHIZHANG.FLF_TARGETS flf
#   ON dd.STORE_STARTING_POINT_ID = flf.STARTING_POINT_ID
#   AND hour(convert_timezone('UTC', dd.TIMEZONE, dd.created_at)) BETWEEN flf.min_hour AND flf.max_hour
#   AND convert_timezone('UTC', dd.TIMEZONE, dd.created_at) BETWEEN flf.TARGET_CREATED_AT AND flf.NEXT_TARGET_CREATED_DATE
# LEFT JOIN PRODDB.PUBLIC.MAINDB_SUBMARKET sm 
#   ON dd.SUBMARKET_ID = sm.ID
# LEFT JOIN public.fact_cx_email_summary fces
#   ON dd.SUBMARKET_ID = fces.SUBMARKET_ID
#   AND convert_timezone('UTC',dd.timezone,
#  dateadd('minute',CAST(floor(date_part('minute',dd.created_at) / 30) * 30 AS INT), date_trunc('hour',dd.created_at))
#  ) = fces.half_hour_local
# //  WHERE CAST(DD.CREATED_AT as DATE) >= dateadd('DAY', -3, TO_TIMESTAMP_NTZ(LOCALTIMESTAMP)) 
#   WHERE dd.created_at BETWEEN '2020-03-16' AND '2020-04-27'
#   AND CAST(DD.CREATED_AT as DATE) < dateadd('DAY', 0, TO_TIMESTAMP_NTZ(LOCALTIMESTAMP))  
#   AND dd.IS_FILTERED_CORE = true 
#   AND dd.IS_ASAP = true 
#   AND dd.IS_CONSUMER_PICKUP = false 
#   AND fulfillment_type != 'merchant_fleet'
# );

# CREATE TEMP TABLE CHIZHANG.FLF_RAW_GROUPED AS(
# SELECT
#     t.created_at_local_date,
#     t.DAYPART,
#     t.STORE_STARTING_POINT_ID,
#     AVG(t.ASAP) AS avg_asap,
#     AVG(t.DAT) AS avg_dat,
#     AVG(t.D2R) AS avg_d2r,
#     AVG(t.IS_FLF_ABOVE_IDEAL) AS avg_is_flf_above_ideal,
#     AVG(t.flf) AS avg_flf,
#     AVG(t.num_opened) AS avg_num_opened,
#     AVG(t.num_delivered) AS avg_num_delivered
# FROM CHIZHANG.FLF_RAW t
# GROUP BY t.created_at_local_date, t.DAYPART, t.STORE_STARTING_POINT_ID
# );

# SELECT 
#     -- t1.CREATED_AT,
#     -- t1.DELIVERY_ID, 
#     -- t1.ACTIVE_DATE, 
#     dayofweek(t1.CREATED_AT_LOCAL) as DAY_OF_WEEK,
#     hour(t1.CREATED_AT_LOCAL) as HOUR_OF_DAY,
#     t1.STORE_STARTING_POINT_ID,
#     t1.SUBMARKET_ID,
#     -- t1.FLF,
#     -- t1.NUM_DELIVERED, 
#     -- t1.NUM_OPENED,
#     -- t1.SUBMARKET_LAUNCH_DATE, 
#     -- t1.CREATED_AT_LOCAL, 
#     -- t1.CREATED_AT_LOCAL_DATE,
#     -- t1.ASAP, 
#     -- t1.DAT,
#     -- t1.D2R, 
#     -- t1.LATENESS_20_MIN,
#     t1.DAYPART,
#     t1.WINDOW_ID
#     -- t1.IS_FLF_ABOVE_MAX, 
#     -- t1.IS_FLF_ABOVE_IDEAL,
#     -- t2.avg_asap,
#     -- t2.avg_dat,
#     -- t2.avg_d2r,
#     -- t2.avg_is_flf_above_ideal,
#     -- t2.avg_flf,
#     -- t2.avg_num_delivered,
#     -- t2.avg_num_opened
# FROM CHIZHANG.FLF_RAW t1
# LEFT JOIN CHIZHANG.FLF_RAW_GROUPED t2
#     ON t1.created_at_local_date = DATEADD(Day, -15, t2.created_at_local_date)
#     AND t1.DAYPART = t2.DAYPART
#     AND t1.STORE_STARTING_POINT_ID = t2.STORE_STARTING_POINT_ID
# SAMPLE(5)
# """
user = dbutils.secrets.get(scope="chizhang-scope", key="snowflake-user")
password = dbutils.secrets.get(scope="chizhang-scope", key="snowflake-password")

os.environ['SNOWFLAKE_USER'] = dbutils.secrets.get(scope="chizhang-scope", key="snowflake-user")
os.environ['SNOWFLAKE_PW'] = dbutils.secrets.get(scope="chizhang-scope", key="snowflake-password")

# snowflake connection options
params = dict(
  user=os.environ['SNOWFLAKE_USER'],
  password=os.environ['SNOWFLAKE_PW'],
  account='DOORDASH',
  database='PRODDB',
  warehouse='ADHOC',
  schema='public',
)

## Data loading

In [8]:
scope_name = 'chizhang-scope'
pw_key_name = 'snowflake-password'
un_key_name = 'snowflake-user'
user = dbutils.secrets.get(scope=scope_name, key=un_key_name)
password = dbutils.secrets.get(scope=scope_name, key=pw_key_name)

# snowflake connection options
options = dict(sfurl="doordash.snowflakecomputing.com/",
               sfaccount="DOORDASH",
               sfuser=user,
               sfpassword=password,
               sfdatabase="PRODDB",
               sfschema="public",
               sfwarehouse="ADHOC")

print(user)
print(password)

[REDACTED]
[REDACTED]

In [9]:
sql_load_table = """ select * from variance_reduction_flf_train_test """
all_data = spark.read.format("snowflake").options(**options).option("query", sql_load_table).load()
raw_data = all_data.toPandas()

In [10]:
# filename = 'percentage.csv'
# filename = 'small.csv'
# filename = 'data_new_agg_feat_sample20.csv'
# filename = 'variation_reduction_new_feat_sample20.csv'

# dbutils.fs.cp('dbfs:/chizhang/variance_reduction/data/{}'.format(filename), 'file:/databricks/driver/{}'.format(filename))
# raw_data = pd.read_csv(filename)
# sql_query = "select * from CHIZHANG.VARIANCE_REDUCTION_FLF"

sql_query = sql_query_train
with snowflake.connector.connect(**params) as ctx:
  raw_data = pd.read_sql(sql_query, ctx)
  raw_data.to_csv('/dbfs/chizhang/variance_reduction/data/data_new_agg_feat_sample20_post_covid.csv')

In [11]:
df = raw_data.copy()
df = df.reindex()
print('data size', df.shape)
df.columns = map(str.lower, df.columns)
df.head(5)

,created_at_local_date,day_of_week,hour_of_day,store_starting_point_id,submarket_id,daypart,window_id,is_flf_above_ideal
0,2020-04-06,1,9,254,9,breakfast,18,0
1,2020-04-06,1,9,255,9,breakfast,18,0
2,2020-04-06,1,9,255,9,breakfast,18,0
3,2020-04-06,1,9,255,9,breakfast,18,1
4,2020-04-06,1,9,254,9,breakfast,18,0


## Feature creating

In [13]:
# combine real-time features with historical aggregated features
# features_cat = ['day_of_week', 'hour_of_day', 'STORE_STARTING_POINT_ID', 'SUBMARKET_ID', 'LATENESS_20_MIN', 'DAYPART', '30min'] #30 min, sp, submarket, ->market level feats
# historical aggregated features := avg(feat_values) over SP_ID/(DATE_LOCAL-15days)/DAYPART units (i.e., 14 days ago) 
# features_num = ['AVG_ASAP', 'AVG_DAT', 'AVG_D2R', 'AVG_FLF', 'AVG_IS_FLF_ABOVE_IDEAL']

features_cat = ['day_of_week', 'hour_of_day', 'store_starting_point_id', 'submarket_id', 'daypart', 'window_id'] #30 min, sp, submarket, ->market level feats
# features_num = ['AVG_ASAP', 'AVG_DAT', 'AVG_D2R', 'AVG_FLF', 'AVG_IS_FLF_ABOVE_IDEAL', 'AVG_NUM_DELIVERED', 'AVG_NUM_OPENED']
features_num = []

features = features_cat + features_num
metrics = ['is_flf_above_ideal']

for f in features_cat:
  df[f] = df[f].astype('category')
for f in features_num:
  df[f] = df[f].astype('float')
  
X = df[features]
y = df[metrics]

## Train/test data preparing

In [15]:
from datetime import datetime, timedelta

df['created_at_local_date'] = pd.to_datetime(df['created_at_local_date'])
date_train_end = df['created_at_local_date'].max() - timedelta(weeks=1)
date_train_start = date_train_end - timedelta(weeks=5)

train_index = df[(df['created_at_local_date'] < str(date_train_end)) & (df['created_at_local_date'] >= str(date_train_start))].index
test_index = df[df['created_at_local_date'] >= str(date_train_end)].index

print('training period:', df.loc[train_index]['created_at_local_date'].min(), df.loc[train_index]['created_at_local_date'].max())
print('testing period:', df.loc[test_index]['created_at_local_date'].min(), df.loc[test_index]['created_at_local_date'].max())


training period: 2020-03-16 00:00:00 2020-04-19 00:00:00
testing period: 2020-04-20 00:00:00 2020-04-27 00:00:00

In [16]:
# check train and test index do not intersects
assert len(train_index.intersection(test_index))==0, 'data leakage'
# check if training and testing data has 7 days a week
assert df.loc[test_index]['day_of_week'].nunique() == 7, 'testing data does not have 7 days a week'
assert df.loc[train_index]['day_of_week'].nunique() == 7, 'training data does not have 7 days a week'

In [17]:
X_train = X.loc[train_index]
y_train = y.loc[train_index]

X_test = X.loc[test_index]
y_test = y.loc[test_index]

print("x_train shape", X_train.shape, 'y_train shape', y_train.shape)
print("x_test shape", X_test.shape, 'y_test shape', y_test.shape)

x_train shape (8775841, 6) y_train shape (8775841, 1)
x_test shape (2384570, 6) y_test shape (2384570, 1)

### Create pipeline & train

In [19]:
# target encoding
teppl = TargetEncoder(cols=['store_starting_point_id', 'submarket_id', 'window_id'])
# clf
clfppl = LGBMClassifier()

# random search
randomParams = {
    'clf__learning_rate': stats.uniform(0.01, 0.1), 
    'clf__num_leaves': stats.randint(16, 512),
    'clf__boosting_type' : ['gbdt', 'dart', 'goss']    
}

ppl = Pipeline([('target_encode', teppl), ('clf', clfppl)])

random_search = RandomizedSearchCV(ppl, randomParams, n_jobs=-1, verbose=2, n_iter=100)
random_search.fit(X_train, y_train)


# ppl = pipeline.set_params(clfppl__learning_rate = random_search.best_params_['learning_rate'],
#                           clfppl__num_leaves = random_search.best_params_['num_leaves'],
#                           clfppl__boosting_type = random_search.best_params_['boosting_type']
#                           )
    
# ppl.fit(X_train, y_train)
    

# clf_random_search.fit(X_train, y_train)
# clfppl = clf_random_search.best_estimator_

# # pack encoder and best clf model into a pipeline
# ppl = Pipeline([('target_encode', teppl), ('lightgbm', clfppl)])
# ppl.set_params(target_encode__cols=['store_starting_point_id', 'submarket_id', 'window_id'],
#               lightgbm__learning_rate = stats.uniform(0.01, 0.1), 
#               lightgbm__num_leaves = stats.randint(16, 256),
#               lightgbm__boosting_type = ['gbdt', 'dart', 'goss']).fit(X_train, y_train)

# ppl.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done 34 tasks | elapsed: 6.8min
[Parallel(n_jobs=-1)]: Done 237 tasks | elapsed: 30.4min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 59.3min finished
/local_disk0/pythonVirtualEnvDirs/virtualEnv-396fa41b-568b-453c-aaca-dad4d2b73d4a/lib/python3.5/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 y = column_or_1d(y, warn=True)
/local_disk0/pythonVirtualEnvDirs/virtualEnv-396fa41b-568b-453c-aaca-dad4d2b73d4a/lib/python3.5/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 y = column_or_1d(y, warn=True)
 Out[ 15 ]: RandomizedSearchCV(cv=None, error_score=nan,
 estimator=Pipeline(memory=None,
 steps=[('target_encode',
 TargetEncoder(cols=['store_starting_point_id',
 'submarket_id',
 'window_id'],
 drop_invariant=False,
 handle_missing='value',
 handle_unknown='value',
 min_samples_leaf=1,
 return_df=True,
 smoothing=1.0,
 verbose=0)),
 ('clf',
 LGBMClassifier(boosting_type='gbdt',
 class_we...
 iid='deprecated', n_iter=100, n_jobs=-1,
 param_distributions={'clf__boosting_type': ['gbdt', 'dart',
 'goss'],
 'clf__learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f3bdbe455c0>,
 'clf__num_leaves': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f3bdbe45cf8>},
 pre_dispatch='2*n_jobs', random_state=None, refit=True,
 return_train_score=False, scoring=None, verbose=2)

In [20]:
random_search.best_params_

Out[ 16 ]: {'clf__learning_rate': 0.06663703478387659,
 'clf__boosting_type': 'gbdt',
 'clf__num_leaves': 455}

In [21]:
ppl_best = random_search.best_estimator_

### Testing

In [23]:
residual_std_m1, _ = eval_model(ppl_best, X_test, y_test)

in gt, pct of ones 0.3075288207098135 pct of zeros 0.6924711792901865
residual std 0.4226332599530733 y_test std 0.46147030797511224
logloss_baseline 17.272199785921817
logloss_test_pred 0.5429674792187813
acc 0.7293285581886881
average_precision_score predicted 0.5639626780596179 average_precision_score all zeros 0.3075288207098135
f1 score 0.34792360188520083
roc_auc_score 0.7466992954620041
conf mat [[0.65711847 0.03535271]
 [0.23531874 0.07221008]]

## Eval raw correlation

In [25]:
for f in ['store_starting_point_id', 'submarket_id', 'window_id']:
  print('feature', f)
  print('\tTrain PearsonrResult', pearsonr(ppl_best.named_steps["target_encode"].transform(X_train)[f].values, y_train.values.reshape(len(y_train), )))
#   print('\tTrain PearsonrResult', pearsonr(X_train_encoded[f].values, y_train.values.reshape(len(y_train), )))#, spearmanr(X_train_encoded[f], y_train))  
#   print('\tTest PearsonrResult', pearsonr(X_test_encoded[f].values, y_test.values.reshape(len(y_test), )))#, spearmanr(X_test_encoded[f], y_test))

feature store_starting_point_id
	Train PearsonrResult (0.309752439357013, 0.0)
feature submarket_id
	Train PearsonrResult (0.2405288876488967, 0.0)
feature window_id
	Train PearsonrResult (0.1947056284116564, 0.0)

In [26]:
with open('/dbfs/chizhang/variance_reduction/model/051620_flf_prediction_variance_reduction_ppl.dill', 'wb') as f:
  dill.dump(ppl_best, f)

In [27]:
##### END ####








## validate correlation in unit level

In [29]:
with open('/dbfs/chizhang/variance_reduction/model/051620_flf_prediction_variance_reduction_ppl.dill', 'rb') as f:
  ppl_best = dill.load(f)

In [30]:
for f in ['store_starting_point_id', 'submarket_id', 'window_id']:
  print('feature', f)
  print('\tTrain PearsonrResult', pearsonr(ppl_best.named_steps["target_encode"].transform(X_test)[f].values, y_test.values.reshape(len(y_test), )))

feature store_starting_point_id
	Train PearsonrResult (0.3165188526792855, 0.0)
feature submarket_id
	Train PearsonrResult (0.2388020573006394, 0.0)
feature window_id
	Train PearsonrResult (0.23488981247182145, 0.0)

In [32]:
df.groupby(['store_starting_point_id', 'daypart']).agg({'is_flf_above_ideal' : 'mean'})

is_flf_above_ideal 
 
 
 store_starting_point_id 
 daypart 
 
 
 
 
 
 6 
 breakfast 
 0.339152 
 
 
 dinner 
 0.396425 
 
 
 early_morning 
 0.201550 
 
 
 latenight 
 0.036980 
 
 
 lunch 
 0.356063 
 
 
 snack 
 0.197766 
 
 
 9 
 breakfast 
 0.073864 
 
 
 dinner 
 0.191677 
 
 
 early_morning 
 0.033592 
 
 
 latenight 
 0.034109 
 
 
 lunch 
 0.006754 
 
 
 snack 
 0.088283 
 
 
 10 
 breakfast 
 0.267241 
 
 
 dinner 
 0.380922 
 
 
 early_morning 
 0.229508 
 
 
 latenight 
 0.048673 
 
 
 lunch 
 0.387824 
 
 
 snack 
 0.101695 
 
 
 11 
 breakfast 
 0.085427 
 
 
 dinner 
 0.389990 
 
 
 early_morning 
 0.231579 
 
 
 latenight 
 0.036257 
 
 
 lunch 
 0.106682 
 
 
 snack 
 0.182515 
 
 
 12 
 breakfast 
 0.285470 
 
 
 dinner 
 0.569812 
 
 
 early_morning 
 0.118012 
 
 
 latenight 
 0.044199 
 
 
 lunch 
 0.344269 
 
 
 snack 
 0.164179 
 
 
 ... 
 ... 
 ... 
 
 
 7845 
 snack 
 0.000000 
 
 
 7846 
 breakfast 
 0.000000 
 
 
 dinner 
 0.000000 
 
 
 latenight 
 0.000000 
 
 
 lunch 
 0.000000 
 
 
 snack 
 0.000000 
 
 
 7877 
 breakfast 
 0.000000 
 
 
 dinner 
 0.164179 
 
 
 latenight 
 0.000000 
 
 
 lunch 
 0.000000 
 
 
 snack 
 0.000000 
 
 
 7910 
 breakfast 
 1.000000 
 
 
 dinner 
 0.000000 
 
 
 latenight 
 0.000000 
 
 
 lunch 
 0.000000 
 
 
 snack 
 0.000000 
 
 
 7943 
 breakfast 
 0.236842 
 
 
 dinner 
 0.168675 
 
 
 lunch 
 0.182796 
 
 
 snack 
 0.171429 
 
 
 7976 
 breakfast 
 1.000000 
 
 
 dinner 
 0.875000 
 
 
 latenight 
 1.000000 
 
 
 lunch 
 0.333333 
 
 
 snack 
 0.500000 
 
 
 8011 
 breakfast 
 0.000000 
 
 
 dinner 
 0.000000 
 
 
 latenight 
 0.000000 
 
 
 lunch 
 0.034483 
 
 
 snack 
 0.103448 
 
 
 
 10669 rows × 1 columns

## Model training

In [34]:
# # features_high_corr = ['STORE_STARTING_POINT_ID', 'SUBMARKET_ID', 'hour_of_day', 'day_of_week'] #better residual std without day_of_week

# features_high_corr = features
# #model 1: default with balancing
# clf_default_is_unbalanced = LGBMClassifier(is_unbalance =True)#is_unbalance =True
# clf_default_is_unbalanced.fit(X_train_encoded[features_high_corr], y_train)

# #model 2: default
# clf_default = LGBMClassifier()
# clf_default.fit(X_train_encoded[features_high_corr], y_train)

In [35]:
# # model 3: grid search
# gridParams = {
#     'learning_rate': [0.01, 0.05, 0.1],
#     'num_leaves': [16, 31, 64, 128],
#     'boosting_type' : ['gbdt', 'dart', 'goss'],
#     'objective' : ['binary'],
#     'max_depth' : [-1, 5, 10],
# #     'random_state' : [501], 
# #     'colsample_bytree' : [0.5,0.7],
# #     'subsample' : [0.5,0.7],
# #     'min_split_gain' : [0.01],
#     'min_data_in_leaf':[5, 10, 20],
# #     'metric':['auc']
#     }
# #{'boosting_type': 'goss', 'objective': 'binary', 'learning_rate': 0.1, 'num_leaves': 64, 'max_depth': -1}

# clf_tuned = LGBMClassifier()
# grid = GridSearchCV(clf_tuned, gridParams,verbose=2, n_jobs=-1)
# grid.fit(X_train_encoded[features_high_corr], y_train)

# # print(grid.best_score_)
# print(grid.best_params_)


In [36]:
# # model 4: random search
# import scipy.stats as stats
# randomParams = {
#     'learning_rate': stats.uniform(0.01, 0.1), 
#     'num_leaves': stats.randint(16, 256),
#     'boosting_type' : ['gbdt', 'dart', 'goss'],
#     'objective' : ['binary'],
#     'max_depth' : [-1, 5, 10],
# #     'random_state' : [501], 
# #     'colsample_bytree' : [0.5,0.7],
# #     'subsample' : [0.5,0.7],
# #     'min_split_gain' : [0.01],
# #     'min_data_in_leaf':[10],
# #     'metric':['auc']
#     }
# #{'boosting_type': 'goss', 'objective': 'binary', 'learning_rate': 0.1, 'num_leaves': 64, 'max_depth': -1}

# clf_tuned_random = LGBMClassifier()
# model_random_search = RandomizedSearchCV(clf_tuned_random, randomParams, verbose=2, n_jobs=-1, n_iter=100)
# model_random_search.fit(X_train_encoded[features_high_corr], y_train)

# # print(grid.best_score_)
# print(model_random_search.best_params_)


## Testing

In [38]:
residual_std_m1, _ = eval_model(clf_default_is_unbalanced, X_test_encoded[features_high_corr], y_test)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-148891> in <module> () 
 ----> 1 residual_std_m1 , _ = eval_model ( clf_default_is_unbalanced , X_test_encoded [ features_high_corr ] , y_test ) 

 NameError : name 'clf_default_is_unbalanced' is not defined

In [39]:
residual_std_m2, _ = eval_model(clf_default, X_test_encoded[features_high_corr], y_test)

In [40]:
residual_std_m3, _ = eval_model(grid, X_test_encoded[features_high_corr], y_test)

In [41]:
residual_std_m4, y_std = eval_model(model_random_search, X_test_encoded[features_high_corr], y_test)

## Result plotting

In [43]:
from matplotlib import pyplot as plt
import matplotlib.ticker as mtick

# plot 4 model performance
# std_clf_is_unbalanced = 0.4239141851690449
# std_clf_default = 0.422724398281608
# std_grid_search = 0.4211761747709921
# std_random_search = 0.42077421076528226
# std_random_search_new = 0.4200526230592615 #new features
# std_testing_data = 0.4555727985659514

std_testing_data = y_std

std_clf_is_unbalanced = residual_std_m1
std_clf_default = residual_std_m2
std_grid_search = residual_std_m3
std_random_search = residual_std_m4

heights = [std_testing_data, std_clf_is_unbalanced, std_clf_default, std_grid_search, std_random_search]
heights =[i for i in heights]
heights_pct = [1 - i/std_testing_data for i in heights]
fig, ax = plt.subplots(figsize=(8, 6))
ax.bar(range(len(heights)), heights)
ax.set_ylabel('STD')

plt.xticks(range(len(heights)), ['y std', 'residual_m1 (w/ balancing)', 'residual_m2(w/o balancing)', 'residual_m3(w/ grid search)', 'residual_m4(w/ random search)'])
plt.xticks(rotation=10)
ax2 = ax.twinx()
ax2.plot(heights_pct, '-o', color='black')
ax2.set_ylabel('% of STD Reduction')

vals = ax2.get_yticks()
ax2.set_yticklabels(['{:,.2%}'.format(x) for x in vals])

for i,j in zip(range(len(heights_pct)), heights_pct):
    ax2.annotate('{:,.2%}'.format(j),xy=(i,j))
plt.grid()
display(fig)